In [1]:
import gensim.downloader as api

model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [14]:
model.vocab['весь'] = model.vocab.pop('весь_DET')

In [20]:
keys = list(model.vocab.keys())
for word in keys:
    model.vocab[word[:word.find('_')]] = model.vocab.pop(word)

In [46]:
import pandas as pd
import numpy as np

In [219]:
stories = pd.read_csv('stories_ImagesTexts.csv', header=0, sep=',')

In [220]:
stories.head()

,story_id,images,text
0,127,['https://static2.tinkoff.ru/portfolio/stories...,Как изменить ПИН-код Изменить пин-код Введите ...
1,254,['https://static2.tinkoff.ru/portfolio/stories...,В приложении Тинькофф Как пополнить карту «Стр...
2,865,['https://static2.tinkoff.ru/portfolio/stories...,Тинькофф Инвестиции 2.0 У нас хорошие новости ...
3,1491,['https://static2.tinkoff.ru/portfolio/stories...,"Персонализация — глобальный фэшн-тренд, от кот..."
4,598,['https://static2.tinkoff.ru/portfolio/stories...,Где поиграть Где попробовать Где попрыгать Нео...


In [221]:
stories.drop('images', axis=1, inplace=True)

In [222]:
stories.loc[1, 'text'].replace('\xa0', ' ')

'В приложении Тинькофф Как пополнить карту «Стрелка» Введите номер карты и сумму, которую хотите внести на счет. Нажмите кнопку «Оплатить».  Пополнить карту Откройте вкладку «Оплатить» и выберите в платежах «Транспорт». Выберите в меню карту «Стрелка». Пополнить «Стрелку»'

In [223]:
stories.text[0].replace('\xa0', ' ').lower()

'как изменить пин-код изменить пин-код введите дату своего рождения, чтобы разрешить операцию. зайдите с главной в меню «еще» и выберите во вкладке настроек «пин-коды карт». нажмите «изменить пин-код». введите новую комбинацию и нажмите кнопку «задать пин-код».'

In [89]:
import re

In [178]:
re.sub('[«».,\(\);]', ' ', stories.text[124].replace('\xa0', ' ').replace('\\n', '').lower()).strip().split()

['café',
 'central',
 'stubenring',
 '24',
 'people',
 'on',
 'caffeine',
 'gumpendorfer',
 'str',
 '11',
 'альтернативная',
 'кофейня',
 'в',
 'здании',
 'церкви',
 'на',
 'месте',
 'морга',
 'где',
 'когда-то',
 'бальзамировали',
 'труп',
 'бетховена',
 'лучший',
 'кофе',
 'в',
 'городе',
 'и',
 'самый',
 'приветливый',
 'бариста',
 'будьте',
 'готовы',
 'что',
 'вам',
 'станут',
 'объяснять',
 'чем',
 'друг',
 'от',
 'друга',
 'отличаются',
 'зерна',
 'из',
 'африки',
 'и',
 'латинской',
 'америки',
 'кофейни',
 'уже',
 'больше',
 'века',
 'играют',
 'заметную',
 'роль',
 'в',
 'культурной',
 'жизни',
 'австрии',
 'венцы',
 'называют',
 'их',
 'вторым',
 'домом',
 'и',
 'проводят',
 'там',
 'дни',
 'и',
 'ночи',
 'напролет',
 'любимое',
 'кафе',
 'зигмунда',
 'фрейда',
 'и',
 'льва',
 'троцкого',
 'в',
 'сердце',
 'старого',
 'города',
 'дорого',
 'но',
 'того',
 'стоит',
 'не',
 'обращайте',
 'внимание',
 'на',
 'высокомерных',
 'официантов',
 'во',
 'фраках',
 '—',
 'это',
 'часть

In [147]:
import pymorphy2

In [148]:
morph = pymorphy2.MorphAnalyzer()

In [161]:
morph.parse('тинькофф-журнала')[0].normal_form

'тинькофф-журнал'

In [167]:
'интеграл' in model

True

In [224]:
def text_proc(text):
    text = text.replace('\xa0', ' ').replace('\\n', '').lower()
    text = re.sub('[«».,\(\);:\'\"]', ' ', text).strip().split()
    text = list(filter(str.isalpha, text))
    text = list(map(lambda x: morph.parse(x)[0].normal_form, text))
    vecs = np.array([model[word] for word in text if word in model])
    return vecs.T.mean(axis=1)

In [225]:
text_proc(stories.text[0]).shape

(300,)

In [226]:
stories['vec'] = stories.text.apply(text_proc)

In [227]:
stories[['coord'+str(a) for a in range(300)]] = pd.DataFrame(stories.vec.values.tolist(), index=stories.index)

In [228]:
stories

,story_id,text,vec,coord0,coord1,coord2,coord3,coord4,coord5,coord6,...,coord290,coord291,coord292,coord293,coord294,coord295,coord296,coord297,coord298,coord299
0,127,Как изменить ПИН-код Изменить пин-код Введите ...,"[0.023945762, 0.027338985, -0.03612542, 0.0142...",0.023946,0.027339,-0.036125,0.014271,-0.022873,0.058570,0.009791,...,-0.021669,0.002791,-0.012763,0.022205,0.006222,-0.018878,-0.007339,0.010592,-0.005058,-0.042888
1,254,В приложении Тинькофф Как пополнить карту «Стр...,"[0.040181585, -0.01047726, -0.03939123, 0.0036...",0.040182,-0.010477,-0.039391,0.003636,-0.037800,0.060992,0.014580,...,-0.017745,-0.004105,-0.019211,-0.006725,-0.022437,0.011936,-0.035963,-0.002342,-0.032582,-0.029871
2,865,Тинькофф Инвестиции 2.0 У нас хорошие новости ...,"[0.014239149, -0.008429763, -0.009090618, 0.00...",0.014239,-0.008430,-0.009091,0.009896,-0.014337,0.014273,-0.005956,...,-0.006727,0.025462,-0.002612,0.018086,-0.024839,-0.020831,0.040859,-0.000697,-0.020284,-0.007832
3,1491,"Персонализация — глобальный фэшн-тренд, от кот...","[0.0049575623, 0.007709383, 0.015406789, 0.030...",0.004958,0.007709,0.015407,0.030614,-0.018569,0.014779,-0.001456,...,0.004823,0.031571,-0.001666,0.004815,-0.039070,-0.031844,0.021100,0.032551,-0.024545,0.006978
4,598,Где поиграть Где попробовать Где попрыгать Нео...,"[-0.021650435, 0.010285066, 0.020995906, 0.011...",-0.021650,0.010285,0.020996,0.011718,-0.016412,0.007425,0.005012,...,0.004557,-0.000098,0.004034,0.026488,-0.023023,-0.033923,0.007638,0.029296,-0.025973,0.012353
5,202,"Если карта или счет скрыты, их не видно в меню...","[0.011209093, -0.03400961, -0.016864786, 0.000...",0.011209,-0.034010,-0.016865,0.000527,-0.032575,0.027695,0.013867,...,0.005486,0.005798,-0.035563,0.016271,-0.010104,-0.014327,-0.005964,0.005619,-0.042734,-0.016609
6,1405,читатель Тинькофф-журнала Подробный разбор Как...,"[0.020357616, -0.022086253, -0.026632989, 0.00...",0.020358,-0.022086,-0.026633,0.006650,-0.000431,0.023220,-0.013084,...,0.011091,0.055041,-0.015305,0.026453,-0.018149,-0.034658,0.042923,0.021293,-0.009690,-0.024832
7,195,"Гриль-бар с говорящим названием, где непременн...","[0.011858786, -0.00074421044, 0.016429449, 0.0...",0.011859,-0.000744,0.016429,0.033936,-0.002667,0.009113,0.003107,...,0.011915,0.020904,-0.002994,0.033310,-0.032938,-0.056247,0.016872,0.041070,-0.013058,0.008119
8,1524,Рюкзаки для школы Ранец Mag Taller В этот ране...,"[0.01606102, -2.594336e-05, 0.0052000405, 0.04...",0.016061,-0.000026,0.005200,0.049089,-0.013109,-0.015367,0.011084,...,0.014060,0.010065,-0.010968,-0.004862,-0.028155,-0.020467,0.004423,0.008046,-0.005522,-0.005568
9,205,Овердрафт — это такой краткосрочный кредит на ...,"[0.04432633, -0.009394535, -0.019130602, -0.00...",0.044326,-0.009395,-0.019131,-0.006713,-0.026453,0.009383,-0.003786,...,0.006734,0.009721,-0.006845,-0.001141,-0.014073,-0.024375,0.012917,-0.018621,-0.024646,-0.000195


In [229]:
stories.drop(['text', 'vec'], axis=1, inplace=True)

In [230]:
stories.describe()

,story_id,coord0,coord1,coord2,coord3,coord4,coord5,coord6,coord7,coord8,...,coord290,coord291,coord292,coord293,coord294,coord295,coord296,coord297,coord298,coord299
count,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,...,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000
mean,968.581856,0.020077,-0.001394,-0.002279,0.014716,-0.014518,0.010705,0.005636,-0.002853,-0.003751,...,0.001415,0.020566,-0.005877,0.009023,-0.024578,-0.027319,0.024092,0.011769,-0.027138,-0.003627
std,380.270516,0.018142,0.012580,0.017625,0.017316,0.012876,0.013986,0.014797,0.014261,0.013831,...,0.012854,0.015948,0.013059,0.014491,0.011556,0.016012,0.016128,0.020120,0.015437,0.018206
min,123.000000,-0.026230,-0.051302,-0.058670,-0.029212,-0.105030,-0.077254,-0.040422,-0.052833,-0.064638,...,-0.053394,-0.026739,-0.049668,-0.041390,-0.057800,-0.067647,-0.035963,-0.052594,-0.086650,-0.077489
25%,706.500000,0.006902,-0.008288,-0.014385,0.001020,-0.022873,0.001678,-0.003969,-0.011623,-0.012296,...,-0.006382,0.011024,-0.014305,0.000379,-0.032170,-0.038567,0.013716,-0.003079,-0.037102,-0.014972
50%,1016.000000,0.018700,-0.000988,-0.000484,0.014076,-0.014311,0.011027,0.005420,-0.001255,-0.003213,...,0.000793,0.018991,-0.005517,0.008619,-0.025721,-0.028990,0.023969,0.011055,-0.026467,-0.002132
75%,1278.000000,0.033006,0.006110,0.009507,0.028431,-0.006209,0.019130,0.014304,0.006320,0.004846,...,0.009460,0.027130,0.002637,0.016201,-0.017941,-0.018374,0.034660,0.027218,-0.018091,0.007729
max,1577.000000,0.098995,0.039669,0.055182,0.061105,0.033403,0.060992,0.066887,0.035108,0.042149,...,0.046986,0.091734,0.062646,0.083194,0.023802,0.030077,0.078805,0.075987,0.026688,0.051635


In [231]:
stories.to_csv('stories_textvecs.csv', sep=',', index=False)